In [5]:
import os
import sys

# Save current working directory
cwd = os.getcwd()

# Change to parent directory
parent_dir = os.path.abspath(os.path.join(cwd, '..'))
os.chdir(parent_dir)

# Temporarily add parent directory to sys.path
sys.path.insert(0, parent_dir)
import framework
sys.path.pop(0)

# Return to original directory
os.chdir(cwd)

In [6]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import vit_b_32, ViT_B_32_Weights
from torch import nn, optim
from tqdm import tqdm


weights = ViT_B_32_Weights.DEFAULT
# Transform matching ImageNet-trained model
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    weights.transforms()
])


# CIFAR-10 test data
dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=False)


Files already downloaded and verified


In [7]:
from torch import nn

# Load pretrained model
model = vit_b_32(weights=weights)
model.eval()

model.heads = nn.Linear(in_features = 768, out_features = 100, bias = True)

device = "cuda"

In [ ]:
# Datasets & loaders
train_data = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.to(device)


# Fine-tuning loop
for epoch in range(1):  # adjust epochs as needed
    model.train()
    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")
        
    print(f"Epoch {epoch+1}, Average Loss: {running_loss/len(train_loader):.4f}")



In [ ]:
# Evaluation
model.eval()
correct = total = 0
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Top-1 Accuracy on CIFAR-100 (fine-tuned): {correct / total:.4f}")

100%|██████████| 157/157 [00:30<00:00,  5.07it/s]

Top-1 Accuracy on CIFAR-100 (fine-tuned): 0.0348


In [ ]:
# Save only the model weights (recommended)
torch.save(model.state_dict(), 'model_weights.pth')